In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Фиксируем зерно для повторяемости результатов
np.random.seed(0)

# Загрузка датасета
data = pd.read_csv("data.csv")
print (data.head())

# Извлекаем тексты комментариев
comments = data["comment_text"]

# Целевая переменная: 1 — токсичный, если target > 0.7; иначе — 0
target = (data["target"] > 0.7).astype(int)

       id    target                                       comment_text  \
0   59856  0.893617               haha you guys are a bunch of losers.   
1  239607  0.912500  Yet call out all Muslims for the acts of a few...   
2  239612  0.830769  This bitch is nuts. Who would read a book by a...   
3  240311  0.968750                                   You're an idiot.   
4  240329  0.900000  Who cares!? Stark trek and Star Wars fans are ...   

   severe_toxicity   obscene  identity_attack    insult  threat  asian  \
0         0.021277  0.000000         0.021277  0.872340  0.0000    0.0   
1         0.050000  0.237500         0.612500  0.887500  0.1125    0.0   
2         0.107692  0.661538         0.338462  0.830769  0.0000    0.0   
3         0.031250  0.062500         0.000000  0.968750  0.0000    NaN   
4         0.100000  0.200000         0.000000  0.900000  0.0000    NaN   

   atheist  ...  article_id    rating  funny  wow  sad  likes  disagree  \
0      0.0  ...        2006  reject

Задание 1. Разделение на train и test
--
Почему делаем это: чтобы обучать модель на одной части данных, а проверять — на другой, независимой. Это важно для честной оценки качества.

In [6]:
# Разделим данные: 70% для обучения, 30% для теста
X_train, X_test, y_train, y_test = train_test_split(comments, target, test_size=0.3, random_state=0)

# Проверим размеры
print(f"Train: {X_train.shape[0]} комментариев")
print(f"Test: {X_test.shape[0]} комментариев")

Train: 63631 комментариев
Test: 27271 комментариев


Задание 2. Преобразование текста в числа через CountVectorizer
--

Почему CountVectorizer: он создает таблицу, где строки — комментарии, а столбцы — уникальные слова. Значения — сколько раз слово встречается в комментарии.

In [7]:
# Создаем векторизатор, который преобразует текст в числовые векторы по частоте слов
vectorizer = CountVectorizer()

# Обучаем его на тренировочных текстах и применяем к train и test
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Посмотрим форму полученной матрицы признаков
print(f"Количество признаков: {X_train_vec.shape[1]}")

Количество признаков: 58152


Задание 3. Логистическая регрессия и accuracy
--
Почему логистическая регрессия: это простая и интерпретируемая модель, часто используется как baseline.

In [8]:
# Обучаем логистическую регрессию с увеличенным количеством итераций
classifier = LogisticRegression(max_iter=2000)
classifier.fit(X_train_vec, y_train)

# Предсказания на тестовой выборке
y_pred = classifier.predict(X_test_vec)

# Accuracy — доля правильно классифицированных комментариев
acc = accuracy_score(y_test, y_pred)
print(f"Точность (accuracy) на тесте: {acc:.4f}")


Точность (accuracy) на тесте: 0.9285


Задание 4. Функция предсказания токсичности по тексту
--
позволяет в реальном времени проверять, как модель реагирует на новые тексты.

In [30]:
def predict_toxicity(comment):
    vec = vectorizer.transform([comment])
    prob = classifier.predict_proba(vec)[0][1]  # вероятность токсичности
    return prob

# Пример
print(f"Токсичность: {predict_toxicity('abomination'):.3f}")

Токсичность: 0.374


Задание 5–6. Примеры предсказания на "apples" + вывод 10 самых токсичных слов
--
модель принимает решение на основе веса каждого слова. Мы можем увидеть, какие слова она считает "токсичными" по статистике.

In [18]:
# Проверка конкретных комментариев
print("Apples are stupid:", predict_toxicity("Apples are stupid"))
print("I love apples:", predict_toxicity("I love apples"))

# Получаем веса признаков (слов) от модели
feature_names = np.array(vectorizer.get_feature_names_out())
weights = classifier.coef_[0]

# Находим индексы самых "токсичных" слов — те, у которых вес наибольший
top_toxic_ids = np.argsort(weights)[-10:][::-1]
top_toxic_words = feature_names[top_toxic_ids]
top_weights = weights[top_toxic_ids]

# Выводим 10 самых токсичных слов
print("\nТоп-10 слов, считающихся токсичными моделью:")
for word, weight in zip(top_toxic_words, top_weights):
    print(f"{word}: {weight:.4f}")

Apples are stupid: 0.9994811959875193
I love apples: 0.09864710468625032

Топ-10 слов, считающихся токсичными моделью:
stupid: 9.5496
idiot: 8.6762
idiots: 8.6292
stupidity: 7.5847
idiotic: 6.8082
crap: 6.5476
dumb: 6.5281
pathetic: 6.4570
morons: 6.3474
moron: 6.3433


Задание 7. Интерпретация результата (комментарий)
--



Слова в топе токсичности действительно являются оскорбительными.

---
Но, есть и нейтральные слова — это может указывать на смещение в данных или эффект контекста, возможно если мы доработаем связь контекста, слова как "crap" не будет в ТОП- 10.

---
Я проверил слова которые в US считаются ЭКСТРИМАЛЬНО токсичными, тут они выдают результаты не выше 0.4 - Что мне кажется подозрительно


Задание 8. Проверка bias на чувствительные фразы
--

проверка, есть ли у модели предвзятость (bias) по религии или расе — важный шаг в этике AI.

In [40]:
test_phrases = [
    "\nI have a muslim friend",
    "I have a christian friend",
    "I have an indian friend",
    "I have a jew friend",
    "\nI have a black friend",
    "I am proud to be black",
    "\nI have a homosexual friend",
    "I have a lesbian friend",
    "I have a bisexual friend",
    "I have a transgender friend",
    "I have a agender friend"
]

print("\nBias-тест на чувствительные фразы:")
for phrase in test_phrases:
    score = predict_toxicity(phrase)
    print(f"{phrase:35s} → токсичность: {score:.3f}")


Bias-тест на чувствительные фразы:

I have a muslim friend             → токсичность: 0.445
I have a christian friend           → токсичность: 0.122
I have an indian friend             → токсичность: 0.196
I have a jew friend                 → токсичность: 0.258

I have a black friend              → токсичность: 0.515
I am proud to be black              → токсичность: 0.495

I have a homosexual friend         → токсичность: 0.453
I have a lesbian friend             → токсичность: 0.347
I have a bisexual friend            → токсичность: 0.215
I have a transgender friend         → токсичность: 0.416
I have a agender friend             → токсичность: 0.123


Задание 9. Теоретический вывод
--



Если модель систематически определяет, что комментарии с упоминанием "muslim", "black", "homosexual, "transgender" и других идентичностей более токсичны, это указывает на наличие предвзятости (bias), которая заложена ещё на этапе разметки или сбора данных.

Такой тип предвзятости можно отнести в первую очередь к Групповой по неосведомлённости (Fairness through unawareness), потому что даже если в обучении модели явно не использовалась информация о религии, расе или ориентации, прокси-признаки (например, сами слова "muslim", "homosexual", "black") всё ещё остаются в тексте.

Таким образом, модель неявно учится ассоциировать упоминание определённых групп с токсичностью, даже если сами комментарии нейтральные или положительные.
Кроме того, здесь можно наблюдать проявление исторической предвзятости (historical bias), когда данные отражают и закрепляют существующие в обществе стереотипы и несправедливости, а не исправляют их. Даже если мы строим идеальный алгоритм на "грязных" исторических данных, он будет повторять ошибки прошлого.

Важно понимать, что модель в таком случае не просто ошибается — она усиливает уже существующие социальные дисбалансы о которых мы говорили на уроках. А это уже серьёзный этический риск при применении в реальных системах.

---
Таким образом, при проектировании моделей машинного обучения важно стремиться не только к высокой точности предсказаний,  
но и уделять особое внимание выявлению и исправлению потенциальных источников несправедливости и предвзятости.  
Только так можно создать действительно этичные и надёжные алгоритмы, которые будут служить всем пользователям без дискриминации.

Задание 10. Как улучшить этичность модели
---

---
- Применить более сложные модели, обученные с регуляризацией fairness-aware (например, с ограничениями на дисбаланс TPR).
---
- Ввести двойную модель: одна предсказывает токсичность, другая — "уверенность в нетоксичности".
Итоговое решение принимается на основе объединения предсказаний двух моделей:

если обе уверены, что токсично — ок, токсично.

если разногласие — отметить комментарий как "неуверенный" и отправить на доанализ.

---

- Вместо бинарного "токсичен/нет", сначала классифицировать комментарий по типу токсичности:

              insult (оскорбление),

              threat (угроза),

              identity_attack (нападение на группу),

              и только потом — финальная токсичность.

Строим несколько моделей: одна определяет "угроза или нет", другая — "атака на идентичность", и т.д.

Финальное решение комбинируем.

---
 - Сила токсичности как дополнительный сигнал


Использовать колонку severe_toxicity как вес к целевой переменной.

Например: если обычная токсичность target > 0.7, но severe_toxicity тоже высоко - усиливаем уверенность в предсказании.

---